In [ ]:
!pip install spacy -q
!python -m spacy download en_core_web_sm -q


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-08 08:59:02.897542: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 08:59:05.756721: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-08 08:59:05.756890: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open share

In [ ]:
#import en_core_web_lg # Large SpaCy model for English language
import numpy as np
import re # regular expressions
import spacy # NLU library

from collections import defaultdict
from sklearn.svm import SVC # Support Vector Classification model

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nlp = spacy.load("en_core_web_sm")


#data

In [ ]:
# Create training data
training_sentences = [
    "Hello!", "Hello", "Hi!", "Hello !",
    "Hi",
    "What are some strategies for playing defense in a football match when using a high block tactic?",
    "When implementing a high block tactic in a football game, what are some defensive techniques to employ?",
    "In a football match, when utilizing a high block tactic, what are the recommended defensive measures to take?",
    "What defensive maneuvers should be executed when using a high block tactic in a football game?",
    "When playing with a high block tactic in football, how can the defense effectively protect their goal?", 
    "When a team has a defensive identity, what offensive strategies should they employ?",
    "How should a team with a defensive identity approach their offensive game plan?",
    "In the context of a team's defensive identity, what offensive styles are recommended?",
    "What offensive tactics should a team with a defensive identity utilize?",
    "When a team's identity is based on defense, what offensive techniques should they incorporate into their play?",
    "Thank you", "thanks", "goodbye","thanks a lot", "thank you very much"
    
]
training_intents = [
    "greeting",
    "greeting",
    "greeting",
    "greeting",
    "greeting",
    "press",
    "press",
    "press",
    "press",
    "press",
    "counter_attack",
    "counter_attack",
    "counter_attack",
    "counter_attack",
    "counter_attack",
    "end", "end", "end", "end", "end"  
]

In [ ]:
tactic_explained={"press": "I recommend you the high pressure tactic which refers to a defensive strategy in which a team attempts to regain possession of the ball quickly by putting intense pressure on the opposing team's players when they are in possession of the ball, often in their own half of the pitch. This is typically done by the team's forwards and midfielders, who work together to cut off passing options, close down space, and force mistakes.The objective of high pressure tactic is to disrupt the opposing team's rhythm, limit their time on the ball, and force turnovers in dangerous areas of the pitch, which can lead to scoring opportunities. However, high pressure tactics require a high level of fitness, coordination, and discipline from the players, as it can be physically demanding and requires a lot of teamwork to execute effectively.",
                  "counter_attack": "I recommend you counter-attacking tactic which is a tactical strategy used by teams that prefer to defend deep and quickly transition to attack when they win the ball back.The idea behind the counter-attack tactic is to catch the opposition off-guard and take advantage of the open spaces that are left behind when the opposing team commits players forward. When a team wins the ball back in their own half, the defenders or midfielders quickly play a long ball or pass to their forwards, who will look to sprint forward and exploit the space that the opposition has left behind.The counter-attack tactic requires fast and skillful forwards who can create chances out of nothing, as well as defenders and midfielders who are disciplined and good at intercepting the ball.",
                  "greeting": "Hi! How can I help you?",
                  "end":"I hope my tips were useful to you! Wish all the best and never hesitate to come again if you have further questions. Johan Cruyf is always happy to help ^^",
                  }

# DATA PREPROCESSING

In [ ]:
# Initialize the array with zeros: X
X_train = np.zeros((len(training_sentences), 
              nlp('sentences').vector.size))

for i, sentence in enumerate(training_sentences):
    # Pass each each sentence to the nlp object to create a document
    doc = nlp(sentence)
    #print(doc.vector.shape)
    # Save the document's .vector attribute to the corresponding row in X
    X_train[i, :] = doc.vector

In [ ]:
X_train.shape

(20, 96)

# train

In [ ]:
# Create a support vector classifier
clf = SVC(C=1, gamma="auto", probability=True)

# Fit the classifier using the training data
clf.fit(X_train, training_intents)

SVC(C=1, gamma='auto', probability=True)

#Test

In [ ]:
def get_response_ml(text):
    doc = nlp(text)
    return(tactic_explained[clf.predict([doc.vector])[0]])

In [ ]:
get_response_ml("Hello")

'Hi! How can I help you?'

In [ ]:
get_response_ml("how to defend while playing with a high block tactic?")

"I recommend you the high pressure tactic which refers to a defensive strategy in which a team attempts to regain possession of the ball quickly by putting intense pressure on the opposing team's players when they are in possession of the ball, often in their own half of the pitch. This is typically done by the team's forwards and midfielders, who work together to cut off passing options, close down space, and force mistakes.The objective of high pressure tactic is to disrupt the opposing team's rhythm, limit their time on the ball, and force turnovers in dangerous areas of the pitch, which can lead to scoring opportunities. However, high pressure tactics require a high level of fitness, coordination, and discipline from the players, as it can be physically demanding and requires a lot of teamwork to execute effectively."

In [ ]:
get_response_ml("For a team with a defensive idendity, what offensive styles should be adopted?")

'I recommend you counter-attacking tactic which is a tactical strategy used by teams that prefer to defend deep and quickly transition to attack when they win the ball back.The idea behind the counter-attack tactic is to catch the opposition off-guard and take advantage of the open spaces that are left behind when the opposing team commits players forward. When a team wins the ball back in their own half, the defenders or midfielders quickly play a long ball or pass to their forwards, who will look to sprint forward and exploit the space that the opposition has left behind.The counter-attack tactic requires fast and skillful forwards who can create chances out of nothing, as well as defenders and midfielders who are disciplined and good at intercepting the ball.'

In [ ]:
get_response_ml("Thank you")

'I hope my tips were useful to you! Wish all the best and never hesitate to come again if you have further questions. Johan Cruyf is always happy to help ^^'